In [97]:
class Cola:
    turno_actual = 1
    def __init__(self):
        self.citasProcesar = []
        self.tamanio = 0
    def agregarCita(self, cita):
        self.citasProcesar.append(cita)
        self.tamanio += 1
        self.turno_actual += 1
        print("CITA AGREGADA EXITOSAMENTE EN LA COLA DE PENDIENTES")
    def eliminarCita(self, cita):
        self.citasProcesar.remove(cita)
        self.tamanio -= 1
    def reincorporarCita(self, pos, cita):
        self.citasProcesar.insert(pos,cita)
        self.tamanio += 1
    def dequeCita(self):
        if self.tamanio == 0:
            return
        else:
            self.tamanio -= 1
            return self.citasProcesar.pop(0)
    def front(self):
        if self.tamanio == 0:
            return None
        else:
            return self.citasProcesar[0]
    def recorrerCola(self):
        if self.tamanio == 0:
            print("NO HAY CITAS PENDIENTES EN LA COLA")
            return
        else:
            for cita in self.citasProcesar:
                print(f'CITA: {cita}')

In [98]:
class Nodo:
    def __init__(self, valor):
        self.valor = valor
        self.siguiente = None
    def __str__(self):
        return f'{self.valor}'

In [99]:
class ListaEnlazada:
	def __init__(self):
		self.nodoCabeza = None
		self.tamanio = 0
	def push_back(self,valor):
		nuevoNodo = Nodo(valor)
		if self.nodoCabeza == None:
			self.nodoCabeza = nuevoNodo
		else:
			nodoActual = self.nodoCabeza
			while nodoActual.siguiente != None:
				nodoActual = nodoActual.siguiente
			nodoActual.siguiente = nuevoNodo
		self.tamanio += 1
	def recorrerLista(self):
		if self.nodoCabeza != None:
			nodoActual = self.nodoCabeza
			while nodoActual != None:
				print(nodoActual)
				nodoActual = nodoActual.siguiente
		else:
			print("NO HAY CITAS REGISTRADAS")
	def deleteNodo(self, pos):
		if self.nodoCabeza == None:
			return False
		if pos > self.tamanio:
			return False
		if pos == 0 and self.tamanio == 1:
			self.nodoCabeza = None
		elif pos == 0:
			siguiente = self.nodoCabeza.siguiente
			self.nodoCabeza = siguiente
			self.tamanio -= 1
		else:
			nodoPrevio = self.nodoCabeza
			nodoSiguiente = self.nodoCabeza.siguiente
			for _ in range(pos-1):
				nodoPrevio = nodoSiguiente
				nodoSiguiente = nodoSiguiente.siguiente
				if(nodoSiguiente == None):
					nodoPrevio.siguiente = None
				else:
					nodoPrevio.siguiente = nodoSiguiente.siguiente
					self.tamanio -= 1
	def deleteNodoById(self, id):
		if self.nodoCabeza == None:
			print("NO HAY CITAS REGISTRADAS")
			return
		else:
			contadorTam = 0
			nodoActual = self.nodoCabeza
			while nodoActual != None:
				if nodoActual.valor.id == id:
					self.deleteNodo(contadorTam)
					print("CITA ELIMINADA EXITOSAMENTE")
				nodoActual = nodoActual.siguiente
				contadorTam += 1
	def findCitaById(self, id):
		if self.nodoCabeza == None:
			return None
		else:
			nodoActual = self.nodoCabeza
			while nodoActual != None:
				if nodoActual.valor.id == id:
					return nodoActual.valor
				nodoActual = nodoActual.siguiente
			return None

In [100]:
class PilaHistorial():
    def __init__(self):
        self.operacionesRegistradas = []
        self.tamanioPila = 0

    def agregarOperacion(self, cita, tipoOperacion):
        self.operacionesRegistradas.append((cita, tipoOperacion))
        self.tamanioPila += 1

    def pop(self):
        self.tamanioPila -= 1
        return self.operacionesRegistradas.pop()

    def peek(self):
        if self.tamanioPila == 0:
            return (None, None)
        return (self.operacionesRegistradas[self.tamanioPila-1][0],self.operacionesRegistradas[self.tamanioPila-1][1])
    def size(self):
        return self.tamanioPila


In [101]:
class Cita:
    id_counter = 1
    
    def __init__(self, nombrePaciente, hora, nombreDoctor, estado, turno):
        self.id = Cita.id_counter
        Cita.id_counter += 1
        self.nombrePaciente = nombrePaciente
        self.hora = hora
        self.nombreDoctor = nombreDoctor
        self.estado = estado
        self.turno = turno
        
    def __str__(self):
        return f"ID: {self.id} | Paciente: {self.nombrePaciente} | Hora: {self.hora} | Doctor: {self.nombreDoctor} | Estado: {self.estado} | Turno: {self.turno}"
    

In [102]:
cola_pendientes = Cola()
lista_citas = ListaEnlazada()
historial_modificaciones = PilaHistorial()
def procesar_cita():
    if cola_pendientes.tamanio == 0:
        print("NO HAY CITAS PARA PROCESAR")
        return
    else:
        cita_procesada = cola_pendientes.dequeCita()
        cita_procesada.estado = "COMPLETADA"
        historial_modificaciones.agregarOperacion(cita_procesada, "PROCESAR")
        print("CITA PROCESADA EXITOSAMENTE")
def revertir_operacion():
    if historial_modificaciones.tamanioPila == 0:
        print("NO HAY OPERACIONES REGISTRADAS EN LA PILA")
    else:
        cita, operacion = historial_modificaciones.peek()
        match operacion:
            case 'REGISTRO':
                lista_citas.deleteNodoById(cita.id)
                cola_pendientes.eliminarCita(cita)
                cola_pendientes.turno_actual -= 1
            case 'MODIFICAR':
                cita_modificada = lista_citas.findCitaById(cita.id)
                cita_modificada.nombrePaciente = cita.nombrePaciente
                cita_modificada.hora = cita.hora
                cita_modificada.nombreDoctor = cita.nombreDoctor
            case 'CANCELACION':
                cita_revertir = lista_citas.findCitaById(cita.id) 
                cita_revertir = cita
                cita_revertir.estado = "Programada"
                cola_pendientes.reincorporarCita(cita.turno - 1, cita_revertir)
            case 'PROCESAR':
                cita_procesada = lista_citas.findCitaById(cita.id)
                cita_procesada.estado = "Programada"
                cola_pendientes.reincorporarCita(0, cita_procesada)
        historial_modificaciones.pop()
        print("CAMBIOS REVERTIDOS! ESTADO:")
        print("Lista:")
        lista_citas.recorrerLista()
        print("Cola de pendientes: ")
        cola_pendientes.recorrerCola()
        print("Operación Actual: ")
        print(historial_modificaciones.peek())
def cancelar_cita():
    if cola_pendientes.tamanio == 0:
        print("NO HAY CITAS PARA CANCELAR EN EL SISTEMA")
        return
    try:
        id_cita = int(input("INGRESE EL ID DE SU CITA: "))
        citaCancelar = lista_citas.findCitaById(id_cita)
        cola_pendientes.eliminarCita(citaCancelar)
        citaCancelar.estado = "CANCELADA"
        historial_modificaciones.agregarOperacion(citaCancelar, "CANCELACION")
        print("CITA CANCELADA CON EXITO!")
    except ValueError:
        print("ERROR EN LA CANCELACION DE SU CITA")
        return
def registrar_cita():
    paciente = input("Nombre del paciente: ")
    hora = input("Hora de la cita: ")
    doctor = input("Nombre del doctor: ")
    nueva_cita = Cita(paciente, hora, doctor, "Programada",cola_pendientes.turno_actual)
    cola_pendientes.agregarCita(nueva_cita)
    lista_citas.push_back(nueva_cita)
    historial_modificaciones.agregarOperacion(nueva_cita, "REGISTRO")
    print(f"CITA REGISTRADA CON ID: {nueva_cita.id}")
def listar_cita():
    print("LISTA DE CITAS:")
    lista_citas.recorrerLista()
    print("COLA DE PENDIENTES:")
    cola_pendientes.recorrerCola()
def modificar_cita():
    if lista_citas.tamanio == 0:
        print("NO HAY CITAS PARA MODIFICAR!")
        return
    id_input = input("Ingrese el ID de la cita a modificar: ")
    if not id_input.isdigit():
        print("Error: El ID debe ser un número")
        return
    id_buscar = int(id_input)
    nodo_actual = lista_citas.nodoCabeza
    encontrado = False
    while nodo_actual:
        if nodo_actual.valor.id == id_buscar:
            cita = nodo_actual.valor
            print("\nCita encontrada:")
            print(cita)
            copia = Cita(cita.nombrePaciente, cita.hora, cita.nombreDoctor, cita.estado, cita.turno)
            copia.id = cita.id
            historial_modificaciones.agregarOperacion(copia, "MODIFICAR")
            print("\nOpciones de modificación:")
            print("1. Nombre del paciente")
            print("2. Hora")
            print("3. Doctor")
            opcion = input("Elija qué modificar (1-3): ")
            if opcion == "1":
                cita.nombrePaciente = input("Nuevo nombre: ")
            elif opcion == "2":
                cita.hora = input("Nueva hora: ")
            elif opcion == "3":
                cita.nombreDoctor = input("Nuevo doctor: ")
            else:
                print("Opción no válida")
                return
            print("¡Cambios guardados!")
            encontrado = True
            break  
        nodo_actual = nodo_actual.siguiente 
    if not encontrado:
        print("No existe una cita con ese ID")
print('Sistema de Gestión de Citas Médicas')
while True:
    print('\n1. Registrar cita\n2. Listar citas\n3. Modificar cita\n4. Revertir Cambio\n5. Cancelar Cita\n6. Procesar Cita\n7.Salir')
    opcion = input("Seleccione una opción: ")
    
    if opcion == "1":
        registrar_cita()
        cola_pendientes.recorrerCola()
    elif opcion == "2":
        listar_cita()
    elif opcion == "3":
        modificar_cita()
    elif opcion == '4':
        revertir_operacion()
    elif opcion == '5':
        cancelar_cita()
    elif opcion == '6':
        procesar_cita()
    elif opcion == "7":
        print("Saliendo del sistema...")
        break
    else:
        print("Opción inválida, intente nuevamente")


Sistema de Gestión de Citas Médicas

1. Registrar cita
2. Listar citas
3. Modificar cita
4. Revertir Cambio
5. Cancelar Cita
6. Procesar Cita
7.Salir
NO HAY CITAS PARA MODIFICAR!

1. Registrar cita
2. Listar citas
3. Modificar cita
4. Revertir Cambio
5. Cancelar Cita
6. Procesar Cita
7.Salir
CITA AGREGADA EXITOSAMENTE EN LA COLA DE PENDIENTES
CITA REGISTRADA CON ID: 1
CITA: ID: 1 | Paciente: Paolo | Hora: 3:00 | Doctor: Kami | Estado: Programada | Turno: 1

1. Registrar cita
2. Listar citas
3. Modificar cita
4. Revertir Cambio
5. Cancelar Cita
6. Procesar Cita
7.Salir
No existe una cita con ese ID

1. Registrar cita
2. Listar citas
3. Modificar cita
4. Revertir Cambio
5. Cancelar Cita
6. Procesar Cita
7.Salir

Cita encontrada:
ID: 1 | Paciente: Paolo | Hora: 3:00 | Doctor: Kami | Estado: Programada | Turno: 1

Opciones de modificación:
1. Nombre del paciente
2. Hora
3. Doctor
¡Cambios guardados!

1. Registrar cita
2. Listar citas
3. Modificar cita
4. Revertir Cambio
5. Cancelar Cita
6.